In [1]:
from typing import AnyStr
import os
os.chdir("..")
os.chdir("src")
from data_cleaning import DataCleaning
os.chdir("..")

In [4]:
# https://stackoverflow.com/questions/75280182/topic-modelling-coherence-score

In [2]:
cleaning_pipeline = DataCleaning()
df = cleaning_pipeline.run()

2024-04-01 15:31:36 - INFO - Data loaded!
2024-04-01 15:31:36 - INFO - Null values cleaned!
/home/bruno/mestrado-ufrgs/mestrado-ufrgs-cmp617-tce/src/data_cleaning_pandas.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["ANO_LICITACAO"] = df_final["ANO_LICITACAO"].replace(
2024-04-01 15:31:36 - INFO - Data types asserted!
2024-04-01 15:31:36 - INFO - Full data cleaned!


In [4]:
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

In [13]:
df_trial = df.sample(5000, random_state=42)

In [32]:
docs = df["DS_OBJETO"].values

In [33]:
topic_model = BERTopic(
    nr_topics=5,
    zeroshot_topic_list=[
        "vehicles",
        "construction",
        "health/hospital",
        "education",
        "food",
    ],
    min_topic_size=5,
    language="brazilian portuguese",
    low_memory=True,
)
topics, _ = topic_model.fit_transform(docs)


2024-04-01 15:47:02 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
2024-04-01 15:47:04 - INFO - Use pytorch device_name: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

: 

In [ ]:
df_trial["clusters"] = topics

In [ ]:
df_trial["clusters"].value_counts()

clusters
6    3026
5    1569
7     279
8      38
0      25
1      17
2      17
9      12
3       9
4       8
Name: count, dtype: int64

In [ ]:
cleaned_docs = topic_model._preprocess_text(docs)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topics = topic_model.get_topics()
topics.pop(-1, None)
topic_words = [
[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics
]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
        for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                            texts=tokens, 
                            corpus=corpus,
                            dictionary=dictionary, 
                            coherence='c_v')
coherence = coherence_model.get_coherence()

2024-04-01 15:46:10 - INFO - adding document #0 to Dictionary<0 unique tokens: []>
2024-04-01 15:46:10 - INFO - built Dictionary<10240 unique tokens: ['5mx5m', 'aquisição', 'de', 'medindo', 'piramidal']...> from 5000 documents (total 106326 corpus positions)
2024-04-01 15:46:10 - INFO - Dictionary lifecycle event {'msg': "built Dictionary<10240 unique tokens: ['5mx5m', 'aquisição', 'de', 'medindo', 'piramidal']...> from 5000 documents (total 106326 corpus positions)", 'datetime': '2024-04-01T15:46:10.103140', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-6.5.0-26-generic-x86_64-with-glibc2.35', 'event': 'created'}
2024-04-01 15:46:10 - INFO - using ParallelWordOccurrenceAccumulator<processes=15, batch_size=64> to estimate probabilities from sliding windows
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you ca

In [ ]:
coherence

0.4897245392389076